In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import json
from collections import Counter
import pickle
import numpy as np

from efficientnet_pytorch import EfficientNet

import torch.optim as optim
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.plugins import *

import albumentations as A
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2

from sklearn import metrics, model_selection
from sklearn.model_selection import StratifiedKFold

import optuna
from optuna.integration import PyTorchLightningPruningCallback
os.environ["TORCH_HOME"] = "/media/hdd/Datasets/"

In [2]:
import torchsnooper as sn

# Look at data

In [3]:
main_path = "/media/hdd/Datasets/blindness/"

In [4]:
df = pd.read_csv(main_path+"trainLabels.csv")

In [5]:
df.image = main_path+"trainImages/"+df.image+".jpg"

In [40]:
df.head()

,image,level,kfold
0,/media/hdd/Datasets/blindness/trainImages/3922...,0,0
1,/media/hdd/Datasets/blindness/trainImages/1798...,0,0
2,/media/hdd/Datasets/blindness/trainImages/9857...,0,0
3,/media/hdd/Datasets/blindness/trainImages/2469...,1,0
4,/media/hdd/Datasets/blindness/trainImages/1606...,0,0


In [7]:
df.level.value_counts()

0    25810
2     5292
1     2443
3      873
4      708
Name: level, dtype: int64

In [8]:
df["kfold"] = -1
df = df.sample(frac=1).reset_index(drop=True)
stratify = StratifiedKFold(n_splits=5)
for i, (t_idx, v_idx) in enumerate(
        stratify.split(X=df.image.values, y=df.level.values)):
    df.loc[v_idx, "kfold"] = i
    df.to_csv("train_folds.csv", index=False)

# Create model

In [9]:
# Efficient net b5
class LitModel(pl.LightningModule):
    def __init__(self, num_classes, learning_rate=3e-4, weight_decay=0.0001):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.num_classes = num_classes

        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

        self.enet = EfficientNet.from_pretrained('efficientnet-b5',
                                                 num_classes=self.num_classes)
        in_features = self.enet._fc.in_features
        self.enet._fc = nn.Linear(in_features, num_classes)

#     @sn.snoop()

    def forward(self, x):
        out = self.enet(x)
        return out

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),
                                      lr=self.learning_rate,
                                      weight_decay=self.weight_decay)

        scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=2,
                                                    gamma=0.1)

        return ([optimizer], [scheduler])


#     @sn.snoop()

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch["x"], train_batch["y"]
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        #         loss.requires_grad = True
        acc = accuracy(preds, y)
        self.log('train_acc_step', acc)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch["x"], val_batch["y"]
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        #         loss.requires_grad = True
        acc = accuracy(preds, y)
        self.log('val_acc_step', acc)
        self.log('val_loss', loss)

In [10]:
class ImageClassDs(Dataset):
    def __init__(self,
                 df: pd.DataFrame,
#                  imfolder: str,
                 train: bool = True,
                 transforms=None):
        self.df = df
#         self.imfolder = imfolder
        self.train = train
        self.transforms = transforms

    def __getitem__(self, index):
        im_path = self.df.iloc[index]['image']
        x = cv2.imread(im_path, cv2.IMREAD_COLOR)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)

        if (self.transforms):
            x = self.transforms(image=x)['image']

        y = self.df.iloc[index]['level']
        return {
            "x": x,
            "y": y,
        }

    def __len__(self):
        return len(self.df)

# Load data

In [11]:
class ImDataModule(pl.LightningDataModule):
    def __init__(
            self,
            df,
            batch_size,
            num_classes,
#             data_dir,
            img_size):
        super().__init__()
        self.df = df
#         self.data_dir = data_dir
        self.batch_size = batch_size
        self.train_transform = A.Compose([
            A.RandomResizedCrop(img_size, img_size, p=1.0),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2,
                                 sat_shift_limit=0.2,
                                 val_shift_limit=0.2,
                                 p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1),
                                       contrast_limit=(-0.1, 0.1),
                                       p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                        max_pixel_value=255.0,
                        p=1.0),
            A.CoarseDropout(p=0.5),
            A.Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ],
            p=1.)

        self.valid_transform = A.Compose([
            A.CenterCrop(img_size, img_size, p=1.),
            A.Resize(img_size, img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                        max_pixel_value=255.0,
                        p=1.0),
            ToTensorV2(p=1.0),
        ],
            p=1.)

    def setup(self, stage=None):
        dfx = pd.read_csv("./train_folds.csv")
        train = dfx.loc[dfx["kfold"] != 1]
        val = dfx.loc[dfx["kfold"] == 1]

        self.train_dataset = ImageClassDs(train,
#                                           self.data_dir,
                                          train=True,
                                          transforms=self.train_transform)

        self.valid_dataset = ImageClassDs(val,
#                                           self.data_dir,
                                          train=False,
                                          transforms=self.valid_transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=12,
                          shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.valid_dataset,
                          batch_size=self.batch_size,
                          num_workers=12)

In [12]:
batch_size = 64
num_classes = 5
img_size = 128

In [13]:
dm = ImDataModule(df,
                  batch_size=batch_size,
                  num_classes=num_classes,
                  img_size=img_size)
class_ids = dm.setup()

# Logs

In [14]:
model = LitModel(num_classes)

Loaded pretrained weights for efficientnet-b5


In [15]:
logger = CSVLogger("logs", name="eff-b5")

In [16]:
trainer = pl.Trainer(auto_select_gpus=True,
                     gpus=1,
                     precision=16,
                     profiler=False,
                     max_epochs=1,
                     callbacks=[pl.callbacks.ProgressBar()],
                     enable_pl_optimizer=True,
                     logger=logger,
                     accelerator='ddp',
                     plugins='ddp_sharded')

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
/home/eragon/.local/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: plugin <class 'pytorch_lightning.plugins.sharded_plugin.DDPShardedPlugin'> has added additional required plugins as default: [<class 'pytorch_lightning.plugins.sharded_native_amp_plugin.ShardedNativeAMPPlugin'>]Extend this plugin and override `required_plugins`if this conflicts with your additional plugins.
  warnings.warn(*args, **kwargs)


# Train

In [19]:
trainer.fit(model, dm)


  | Name | Type         | Params
--------------------------------------
0 | enet | EfficientNet | 28.4 M
--------------------------------------
28.4 M    Trainable params
0         Non-trainable params
28.4 M    Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 439it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

# Test

In [24]:
trainer.save_checkpoint('model1.ckpt')

In [28]:
best_checkpoints = trainer.checkpoint_callback.best_model_path

In [29]:
pre_model = LitModel.load_from_checkpoint(checkpoint_path= best_checkpoints).to("cuda")

Loaded pretrained weights for efficientnet-b5


In [30]:
pre_model.eval()
pre_model.freeze()

In [31]:
transforms = A.Compose([
            A.CenterCrop(img_size, img_size, p=1.),
            A.Resize(img_size, img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                        max_pixel_value=255.0,
                        p=1.0),
            ToTensorV2(p=1.0),
        ],
            p=1.)


In [32]:
test_img = transforms(image=cv2.imread("/media/hdd/Datasets/blindness/trainImages/1000_left.jpg"))

In [34]:
y_hat = pre_model(test_img["image"].unsqueeze(0).to("cuda"));y_hat

tensor([[ 2.4552, -0.1713,  0.0672, -2.9802, -2.5015]], device='cuda:0')

In [35]:
int(torch.argmax(y_hat, dim = 1))

0